# Multithreading and Multiprocessing

### Threading
* typically, concurrency is created so that we can do some task while I/O is happening (e.g., a server can start processing a new request while waiting for data from a previous request to arrive)
* we can create objects that appear to be running independently, but simultaneously
* the job of threading is to enable an application to be responsive
* CPython, the default implementation of Python, has a Global Interpreter Lock (GIL), which prevents your application from doing two things at once, but rather, the CPU time is being rationed across your threads

### Simple threading example

In [3]:
from threading import Thread

class InputReader(Thread):
    """Thread example, extends Thread class"""

    def run(self):
        """
        Whatever is in the run method (or called from
        it) is executed in a separate thread
        """
        self.line_of_text = input('Enter some text: ')

input('Are you ready? When you hit return the thread will start.')
thread = InputReader() # create thread object
thread.start() # cf. thread.run() for no concurrency

count, result = 1, 1

while thread.is_alive():
    result = count * count
    count += 1

print(f'calculated squares up to {count:,} * {count:,} = {result:,}')
print(f'while you typed "{thread.line_of_text}"')

Are you ready? When you hit return the thread will start.
Enter some text: hello!
calculated squares up to 1 * 1 = 1
while you typed "hello!"


In [13]:
from threading import Thread
import json
from urllib.request import urlopen
import time

cities = ['Boulder', 'Atlanta', 'San Francisco',
          'Reno', 'Honolulu', 'Zurich', 'Dubai',
          'Dublin', 'Hyderabad', 'Rome']

class TempGetter(Thread):
    def __init__(self, index):
        """Initialize our thread

In the previous example, our class which extended
Thread did not need an __init__ method, because
there was no per-thread information to store. Which
means that the __init__ method from the superclass
(Thread) was called automatically. Here, because we
need to store per-thread information (the city), we
have to explicitly call the__init__ method of Thread.
        """
        super().__init__()
        self.city = cities[index]

    def run(self):
        url_template = (
            'http://api.openweathermap.org/data/2.5/' 
            'weather?q={}&units=imperial'
                        '&&APPID=10d4440bbaa8581bb8da9bd1fbea5617')
        response = urlopen(url_template.format(self.city))
        data = json.loads(response.read().decode())
        self.temperature = data['main']['temp']
        
threads = [TempGetter(c) for c in range(len(cities))] #cities] # creates 10 threads
start = time.time()

# start all 10 threads
for thread in threads:
    thread.start() # not run()

# wait for all 10 threads to complete
for thread in threads:
    thread.join()

for thread in threads:
    print(f"it is {thread.temperature:.0f}°F in {thread.city}")
print(f"Got {len(threads)} temps in {time.time() - start} seconds")

it is 74°F in Boulder
it is 90°F in Atlanta
it is 79°F in San Francisco
it is 75°F in Reno
it is 79°F in Honolulu
it is 56°F in Zurich
it is 90°F in Dubai
it is 61°F in Dublin
it is 73°F in Hyderabad
it is 71°F in Rome
Got 10 temps in 0.1464378833770752 seconds


## Threading (cont'd)
* the main problem with threads is also their primary advantage–shared memory
 * all threads have access to all the memory
 * what if two threads access the same data?
* synchronization is the solution, but it's tricky
 * bugs due to incorrect synchronization can be very difficult to find due to ordering issues
* one solution is to force communication between threads to occur using a data structure that has built in locking, such as queue.Queue
* disadvantages could be outweighed by the fact that shared memory is FAST, except for the GIL

## Repeat A Task Every So Often...
* ...using a Timer (subclass of thread)

In [14]:
from threading import Timer, Event
 
def every_so_often():
    if not done.is_set():
        print('Do the thing you want to every so often')
        Timer(5.0, every_so_often).start()
 
done = Event()
Timer(5.0, every_so_often).start()
 
for count in range(100_000_000):
    prod = count * count
    if count % 10_000_000 == 0:
        print('multiplied up to', count)
 
done.set()

multiplied up to 0
multiplied up to 10000000
multiplied up to 20000000
Do the thing you want to every so often
multiplied up to 30000000
multiplied up to 40000000
multiplied up to 50000000
Do the thing you want to every so often
multiplied up to 60000000
multiplied up to 70000000
Do the thing you want to every so often
multiplied up to 80000000
multiplied up to 90000000


## Lab: threads
* create a program which uses threads to simulate a database server
* your "database server" should simply be a thread which sleeps for a random interval (check out __`time.sleep()`__ and __`random.randint()`__ if you're not familiar with them)
* your main thread should get input from the user and respond to it (perhaps reversing the input given by the user) while the database thread is busy

In [15]:
# Partial solution to above assignment. Added a Queue object which is filled by main program and drained by server.
# Server finishes draining Queue before exiting.

from threading import Thread, Event
import time
import random
from queue import Queue

work_queue = Queue()

class DatabaseServer(Thread):
    def __init__(self):
        super().__init__()
        self.done = Event()
        
    def set_done(self):
        self.done.set()
        # Print message so we know it was received. We won't shut down
        # right away if request remain in Queue.
        print('Database server asked to shut down.')
        
    def run(self):
        """Keep processing requests as long as we weren't told to quit or
           there are requests still in Queue. About 25% of the time we'll
           punt on a request and re-insert it into the Queue.
        """
        while not self.done.is_set() or work_queue.qsize():
            print(f'Queue: {list(work_queue.queue)}')
            secs = work_queue.get()
            if random.random() > 0.75:
                print(f'WARNING: server unable to handle request {secs}...requeuing')
                work_queue.put(secs)
                continue
            print(f'Database server handling request for {secs} seconds')
            time.sleep(secs)
            print('Database server successfully handled request')
        
        print('Database server shutting down.')
        
server = DatabaseServer() # create the thread object
server.start() # start the thread

while True:
    text = input('Enter integer: ')
    if text == 'quit':
        server.set_done()
        break
    print('Enqueueing:', text)
    # Handle case where user does not enter an int. Queue up a
    # random integer rather than the non-int entered by user.
    try:
        seconds = int(text)
    except ValueError:
        seconds = random.randint(10, 15)
        print(f'Invalid integer ({text}), adding {seconds} instead')
    work_queue.put(seconds) # Add request to queue

Queue: []
Enter integer: 3
Enqueueing: 3
Database server handling request for 3 seconds
Enter integer: 5
Enqueueing: 5
Enter integer: 4
Enqueueing: 4
Database server successfully handled request
Queue: [5, 4]
Database server handling request for 5 seconds
Enter integer: 6
Enqueueing: 6
Database server successfully handled request
Queue: [4, 6]
Database server handling request for 4 seconds
Enter integer: 5
Enqueueing: 5
Enter integer: 1
Database server successfully handled request
Queue: [6, 5]
Database server handling request for 6 seconds
Enqueueing: 1
Enter integer: 6
Enqueueing: 6
Enter integer: 7
Database server successfully handled request
Queue: [5, 1, 6]
Database server handling request for 5 seconds
Enqueueing: 7
Enter integer: 8
Enqueueing: 8
Enter integer: 3
Enqueueing: 3
Enter integer: 4
Enqueueing: 4
Database server successfully handled request
Queue: [1, 6, 7, 8, 3, 4]
Database server handling request for 1 seconds
Database server successfully handled request
Queue: [6, 7

# Multiprocessing
* the Python multiprocessing library is designed for cases where CPU-bound jobs needs to happen in parallel and multiple cores are available
* advantages
 * separate memory space for each process
 * code is usually straightforward compared to threads
 * avoids GIL limitation
 * eliminates synchronization (assuming no shared memory)

## A Simple Multiprocessing Example

In [19]:
from multiprocessing import Process, cpu_count
import time
import os

class MuchCPU(Process):
    def run(self):
        print(os.getpid()) # get process ID
        for i in range(80_000_000):
            result = i * i
            
if __name__ == '__main__':
    print('Running...')
    procs = [MuchCPU() for f in range(cpu_count())]
    t = time.time()

    for p in procs:
        p.start()
    
    for p in procs:
        p.join()
    
    print('work took {} seconds'.format(time.time() - t))

Running...
9177
9177
work took 8.964990139007568 seconds


## Multiprocessing (cont'd)
* no reason for more processes than there are processors
 * only `cpu_count()` procs can run simultaneously
 * each proc consumes resources with a full copy of Python interpreter
 * interproc communication is expensive
 * creating procs takes a nonzero amount of time
* so we create at most `cpu_count()` processes when the program starts and have them execute tasks as needed
* easy to implement a basic series of communicating processes to do this, but it can be tricky to debug, test, and get correct–we don't have to do all this work because the Python developers have already done it for us–multiprocessing pools


## Multiprocessing Pools
* pools abstract away the overhead of figuring out what code is running in main process and what code is running in subprocess
* abstraction restricts the number of places that code in different processes interact with each other, making it easier to keep track of
* pools also hide the passing of data between processes
 * using a pool looks much like a function call–you pass data into a function, it's executed in another process or processes, and when the work is complete, a value is returned
 * under the hood, a lot of work is being done to support this–objects in one process are being pickled (serialized) and passed into a pipe, then another process retrieves data from the pipe and unpickles it. Work is done in the subprocess and a result is produced. The result is pickled and passed into a pipe. Eventually, the original process unpickles it and returns it.

## Multiprocessing Pool Example

In [21]:
import random
import math
import os
from multiprocessing.pool import Pool

def prime_factor(value, level=0):
    factors = []
    if level:
        print('    ' * level, 'prime_factor(', value, ', ', level, ') ', os.getpid(), sep='')
    for divisor in range(2, value - 1):
        quotient, remainder = divmod(value, divisor)
        if not remainder:
            factors.extend(prime_factor(divisor, level + 1))
            factors.extend(prime_factor(quotient, level + 1))
            break
    else:
        factors = [value]
    return factors

if __name__ == '__main__': # distiguishes between running and importing
    pool = Pool()

    to_factor = [
        random.randint(40_000_000, 80_000_000) 
                for _ in range(64)
    ]
    print(to_factor)
    results = pool.map(prime_factor, to_factor)
    for value, factors in zip(to_factor, results):
        print("The factors of {} are {}".format(value, factors))
    #print(results)

    prime_factor(2, 1) 11015
    prime_factor(3, 1) 11016
    prime_factor(2, 1) 11017
    prime_factor(293, 1) 11018
    prime_factor(29364877, 1) 11015
    prime_factor(18959759, 1) 11016
    prime_factor(29357010, 1) 11017
    prime_factor(253661, 1) 11018
        prime_factor(7, 2) 11016
        prime_factor(2, 2) 11017
        prime_factor(2708537, 2) 11016
        prime_factor(14678505, 2) 11017
            prime_factor(13, 3) 11016
            prime_factor(3, 3) 11017
            prime_factor(208349, 3) 11016
            prime_factor(4892835, 3) 11017
                prime_factor(89, 4) 11016
                prime_factor(3, 4) 11017
                prime_factor(1630945, 4) 11017
                prime_factor(2341, 4) 11016
                    prime_factor(5, 5) 11017
                    prime_factor(326189, 5) 11017
    prime_factor(7, 1) 11018
    prime_factor(9462601, 1) 11018
[58729754, 65204494, 69270454, 41746948, 56879277, 57264199, 69422356, 70179692, 58714020, 51959163, 4

            prime_factor(6454580, 3) 11015
                prime_factor(2, 4) 11015
                prime_factor(3227290, 4) 11015
                    prime_factor(2, 5) 11015
                    prime_factor(1613645, 5) 11015
                        prime_factor(5, 6) 11015
                        prime_factor(322729, 6) 11015
                            prime_factor(11, 7) 11015
                            prime_factor(29339, 7) 11015
    prime_factor(3, 1) 11015
    prime_factor(23695229, 1) 11015
        prime_factor(17, 2) 11015
        prime_factor(1393837, 2) 11015
    prime_factor(2, 1) 11015
    prime_factor(22578855, 1) 11015
        prime_factor(3, 2) 11015
        prime_factor(7526285, 2) 11015
            prime_factor(5, 3) 11015
            prime_factor(1505257, 3) 11015
                prime_factor(13, 4) 11015
                prime_factor(115789, 4) 11015
                    prime_factor(89, 5) 11015
                    prime_factor(1301, 5) 11015
    prime_factor(2, 1)

## Lab: Multiprocessing Pool
* write a program to compute 1!…48! using a multiprocessing pool
* won't be much of a parallelism example, but it's easy to code
* use previous example as a template

# Multithreading/Multiprocessing for Python 3
* Python 3.2 introduced the __`concurrent.futures`__ module for multithreading  via the ThreadPoolExecutor, or multiprocessing, using ProcessPoolExecutor
* it's been backported to Python 2.6+ and can be installed using __`pip install futures`__

In [22]:
import concurrent.futures
import urllib.request
import time

URLS = ['https://www.japan.go.jp/',
        'http://www.foxnews.com/',
        'http://www.cnn.com/',
        'http://www.python.org',
        'http://www.wikipedia.org',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://www.apple.com',
        'http://blahblahblah.org']

# Retrieve a single page and report the URL and contents
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

start = time.time()
# We use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # start the load operations and mark each future with its URL
    future_to_url = {
        executor.submit(load_url, url, 60): url for url in URLS }
    
    # asynchronously wait for threads to complete...
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print(f'{url} generated an exception: {exc}')
        else:
            print(f'{url} is {len(data)} bytes')
            
print(f'Completed in {time.time() - start:.2f} seconds')

http://www.cnn.com/ generated an exception: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1055)>
http://www.python.org generated an exception: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1055)>
http://www.wikipedia.org generated an exception: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1055)>
http://www.foxnews.com/ generated an exception: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1055)>
http://europe.wsj.com/ generated an exception: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1055)>
https://www.japan.go.jp/ generated an exception: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify faile

In [ ]:
def serial():
    start = time.time()
    for url in URLS:
        try:
            data = load_url(url, 60)
        except Exception as exc:
            print(f'{url} generated an exception: {exc}')
        else:
            print(f'{url} is {len(data)} bytes')
            
    print(f'Completed in {time.time() - start:.2f} seconds')

In [ ]:
serial()

In [23]:
import concurrent.futures
import math

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

def is_prime(n):
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print(f'{number} is prime: {prime}')

In [24]:
%timeit -r 5 main()

112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
112272535095293 is prime: True
112582705942171 is prime: Tru

In [25]:
def serial():
    for prime in PRIMES:
        print(f'{prime} is prime: {is_prime(prime)}')

In [26]:
%timeit -r 5 serial()

112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
112272535095293 is prime: True
112582705942171 is prime: Tru